In [1]:
# ===============================
# Imports
# ===============================
import os
import random
from glob import glob
from tqdm import tqdm

import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [2]:
# ===============================
# Config
# ===============================
DATA_DIR = "/kaggle/input/state-farm-distracted-driver-detection"
TRAIN_DIR = os.path.join(DATA_DIR, "imgs/train")
TEST_DIR  = os.path.join(DATA_DIR, "imgs/test")

IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10
LR = 2e-4
NUM_CLASSES = 10
SEED = 42

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

# ===============================
# Reproducibility
# ===============================
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if DEVICE == "cuda":
    torch.cuda.manual_seed_all(SEED)


Using device: cuda


In [3]:
# List class folders
classes = sorted(os.listdir(TRAIN_DIR))
print("Classes:", classes)

# Count images per class
for cls in classes:
    print(cls, ":", len(os.listdir(os.path.join(TRAIN_DIR, cls))))


Classes: ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
c0 : 2489
c1 : 2267
c2 : 2317
c3 : 2346
c4 : 2326
c5 : 2312
c6 : 2325
c7 : 2002
c8 : 1911
c9 : 2129


In [4]:
filepaths = []
labels = []

for idx, cls in enumerate(classes):
    imgs = glob(os.path.join(TRAIN_DIR, cls, "*.jpg"))
    filepaths.extend(imgs)
    labels.extend([idx] * len(imgs))

df = pd.DataFrame({
    "filepath": filepaths,
    "label": labels
})

print("Total images:", len(df))
df.head()


Total images: 22424


,filepath,label
0,/kaggle/input/state-farm-distracted-driver-det...,0
1,/kaggle/input/state-farm-distracted-driver-det...,0
2,/kaggle/input/state-farm-distracted-driver-det...,0
3,/kaggle/input/state-farm-distracted-driver-det...,0
4,/kaggle/input/state-farm-distracted-driver-det...,0


In [5]:
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label"],
    random_state=SEED
)

print("Train:", len(train_df), "Validation:", len(val_df))


Train: 17939 Validation: 4485


In [6]:
class DriverDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img = Image.open(self.df.loc[idx, "filepath"]).convert("RGB")
        label = self.df.loc[idx, "label"]

        if self.transform:
            img = self.transform(img)

        return img, label


In [7]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2, 0.2, 0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

train_ds = DriverDataset(train_df, train_transform)
val_ds   = DriverDataset(val_df, val_transform)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE,
                          shuffle=False, num_workers=2)


In [8]:
model = models.resnet50(pretrained=True)

# Replace final layer
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 201MB/s]


In [9]:
def train_epoch(model, loader):
    model.train()
    losses, preds, targets = [], [], []

    for imgs, labels in tqdm(loader):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        preds.extend(outputs.argmax(1).cpu().numpy())
        targets.extend(labels.cpu().numpy())

    return np.mean(losses), accuracy_score(targets, preds)


def val_epoch(model, loader):
    model.eval()
    losses, preds, targets = [], [], []

    with torch.no_grad():
        for imgs, labels in tqdm(loader):
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            losses.append(loss.item())
            preds.extend(outputs.argmax(1).cpu().numpy())
            targets.extend(labels.cpu().numpy())

    return np.mean(losses), accuracy_score(targets, preds)


In [10]:
best_acc = 0

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")

    train_loss, train_acc = train_epoch(model, train_loader)
    val_loss, val_acc = val_epoch(model, val_loader)

    scheduler.step()

    print(f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f}")
    print(f"Val   Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("✅ Best model saved")



Epoch 1/10


100%|██████████| 141/141 [00:30<00:00,  4.60it/s]


Train Loss: 0.2366 | Acc: 0.9308
Val   Loss: 0.1060 | Acc: 0.9681
✅ Best model saved

Epoch 2/10


100%|██████████| 141/141 [00:19<00:00,  7.37it/s]


Train Loss: 0.0780 | Acc: 0.9779
Val   Loss: 0.0713 | Acc: 0.9773
✅ Best model saved

Epoch 3/10


...

Train Loss: 0.0049 | Acc: 0.9988
Val   Loss: 0.0189 | Acc: 0.9955
✅ Best model saved

Epoch 9/10


100%|██████████| 141/141 [00:19<00:00,  7.32it/s]


Train Loss: 0.0034 | Acc: 0.9993
Val   Loss: 0.0140 | Acc: 0.9955

Epoch 10/10


100%|██████████| 141/141 [00:18<00:00,  7.61it/s]


Train Loss: 0.0025 | Acc: 0.9993
Val   Loss: 0.0139 | Acc: 0.9962
✅ Best model saved


In [11]:
import torch.nn.functional as F

model.load_state_dict(torch.load("best_model.pth"))
model.eval()

test_images = sorted(glob(os.path.join(TEST_DIR, "*.jpg")))

rows = []

with torch.no_grad():
    for img_path in tqdm(test_images):
        img = Image.open(img_path).convert("RGB")
        img = val_transform(img).unsqueeze(0).to(DEVICE)

        outputs = model(img)
        probs = F.softmax(outputs, dim=1).cpu().numpy()[0]

        row = {
            "img": os.path.basename(img_path)
        }

        for i in range(10):
            row[f"c{i}"] = probs[i]

        rows.append(row)

100%|██████████| 79726/79726 [25:11<00:00, 52.76it/s]


In [12]:
# Create submission dataframe
submission = pd.DataFrame(rows)

# Ensure correct column order
submission = submission[
    ["img", "c0", "c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8", "c9"]
]

submission.to_csv("submission.csv", index=False)
submission.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_1.jpg,3.123705e-04,5.507150e-05,0.000038,2.317703e-05,0.000004,0.998999,8.322701e-06,1.066630e-04,0.000006,0.000448
1,img_10.jpg,7.371075e-05,6.143303e-06,0.000008,5.924924e-06,0.000006,0.999537,5.718516e-06,2.937733e-06,0.000347,0.000008
2,img_100.jpg,8.818904e-01,2.763513e-03,0.003935,1.323799e-02,0.003312,0.001200,4.037151e-04,7.647989e-03,0.023581,0.062029
3,img_1000.jpg,1.258595e-07,6.161636e-07,0.004350,1.316859e-07,0.000005,0.000005,8.525948e-07,4.552830e-07,0.995452,0.000186
4,img_100000.jpg,4.471379e-04,1.022711e-03,0.000006,9.943224e-01,0.000462,0.003618,6.738748e-05,1.706518e-06,0.000044,0.000008
